# Simple Policy

This notebook will show you how to create and run a simple policy

In [ ]:
from vulkan.spec.dependency import INPUT_NODE, Dependency
from vulkan.spec.nodes import BranchNode, TerminateNode
from vulkan.spec.policy import PolicyDefinition

In [ ]:
from lineart import Lineart, models

PROJECT_ID = None
client = Lineart(project_id=PROJECT_ID)

## Creating the Policy

In [ ]:
policy = client.policies.create(
    name="isEven",
    description="A simple policy that checks if a number is even",
)
policy_id = policy.policy_id

In [ ]:
def is_even_fn(input_node):
    if input_node["number"] % 2 == 0:
        return "even"
    return "odd"


branch_node_1 = BranchNode(
    name="branch_node_1",
    func=is_even_fn,
    choices=["even", "odd"],
    dependencies={INPUT_NODE: Dependency(INPUT_NODE)},
)

is_even = TerminateNode(
    name="is_even",
    return_status="Great",
    dependencies={"condition": Dependency("branch_node_1", "even")},
)

is_odd = TerminateNode(
    name="is_odd",
    return_status="Too_Bad",
    dependencies={"condition": Dependency("branch_node_1", "odd")},
)

is_even = PolicyDefinition(
    nodes=[
        branch_node_1,
        is_even,
        is_odd,
    ],
    input_schema={"number": "int"},
)

Create the version `v1` and update it with the definition above

In [ ]:
version_name = "v1"
policy_version = client.policy_versions.create(policy_id=policy_id, alias=version_name)
policy_version_id = policy_version.policy_version_id

In [ ]:
models.WorkflowBase?

In [ ]:
policy_version = client.policy_versions.update(
    alias=version_name,
    policy_version_id=policy_version_id,
    workflow=models.WorkflowBase(
        spec=is_even.to_dict(),
        requirements=[],
    ),
)

## Triggering a Run

In [ ]:
from numpy.random import randint

In [ ]:
run = client.policy_versions.create_run(
    policy_version_id=policy_version_id, input_data={"number": randint(100)}
)

In [ ]:
run_data = client.runs.get_data(run_id=run.run_id)
print(run_data.model_dump_json(indent=4))